In [2]:
import pandas as pd
import altair as alt
from json import loads

G = 1024 * 1024 * 1024
M = 1024 * 1024

In [3]:
df = pd.read_json("../profiling/results/dpvspp.1.json", lines=True)
df["DVM"] = df["mem_size"] / df["memory"]
df["MachineMem"] = df["memory"].apply(lambda value: f"{value / G:.0f}G")
df["DatasetMemSize"] = df["mem_size"].apply(lambda value: f"{value / M:.1f}M")

In [4]:
df.sort_values("mem_size")

,cpu,dataset,fname,format,memory,ncol,nrow,partition,reader,mem_size,name,elapsed,DVM,MachineMem,DatasetMemSize
29,8,heart,create_report_dp.py,csv,68719476736,0,0,0,pandas,36488,DataPrepReport,1.986364,5.309703e-07,64G,0.0M
28,8,heart,create_report_pp.py,csv,68719476736,0,0,0,pandas,36488,PandasProfilingReport,17.657201,5.309703e-07,64G,0.0M
27,8,diabetes,create_report_dp.py,csv,68719476736,0,0,0,pandas,61568,DataPrepReport,1.577283,8.959323e-07,64G,0.1M
26,8,diabetes,create_report_pp.py,csv,68719476736,0,0,0,pandas,61568,PandasProfilingReport,28.295765,8.959323e-07,64G,0.1M
0,8,automobile,create_report_pp.py,csv,68719476736,0,0,0,pandas,219745,PandasProfilingReport,38.194455,3.197711e-06,64G,0.2M
1,8,automobile,create_report_dp.py,csv,68719476736,0,0,0,pandas,219745,DataPrepReport,3.906422,3.197711e-06,64G,0.2M
2,8,titanic,create_report_pp.py,csv,68719476736,0,0,0,pandas,336844,PandasProfilingReport,17.759124,4.901725e-06,64G,0.3M
3,8,titanic,create_report_dp.py,csv,68719476736,0,0,0,pandas,336844,DataPrepReport,2.093762,4.901725e-06,64G,0.3M
10,8,women,create_report_pp.py,csv,68719476736,0,0,0,pandas,2982987,PandasProfilingReport,19.846524,4.340817e-05,64G,2.8M
11,8,women,create_report_dp.py,csv,68719476736,0,0,0,pandas,2982987,DataPrepReport,2.253622,4.340817e-05,64G,2.8M


In [5]:
(
    alt.Chart(df[~df.dataset.isin(["yelp_academic_dataset_review"])], title="create_report(df) Comparison")
    .mark_bar()
    .encode(
        y=alt.Y("name", title=""),
        x=alt.X("elapsed", title="Elapsed (s)"),
        color=alt.Color("name:N", legend=None),
        tooltip=[
            alt.Tooltip("name:N"),
            alt.Tooltip("elapsed:Q", title="Elapsed (s)", format=".2s"),
            alt.Tooltip("MachineMem"),
            alt.Tooltip("DatasetMemSize"),
        ],
        row=alt.Row("dataset", title="Dataset", header=alt.Header(labelAngle=0, labelAlign="left")),
    )
).resolve_scale(x="independent")

alt.Chart(...)

In [ ]:
##
chartDVM = (
    alt.Chart(df[df["DVM"] < 10], title="plot_correlation(df) Comparison")
    .mark_line(point=True)
    .encode(
        y=alt.Y("elapsed", title="Elapsed (s)"),
        x=alt.X("DVM", title="Dataset Size / Memory Size"),
        color="name:N",
        tooltip=[
            alt.Tooltip("name:N"),
            alt.Tooltip("elapsed:Q", format=".0s"),
            alt.Tooltip("MachineMem"),
            alt.Tooltip("DatasetMemSize"),
        ],
        column=alt.Column("format", title="Data Format"),
        row=alt.Column("reader", title="Data Reader"),
    )
)
chartDVM
##
# chartBar = (
#     alt.Chart(
#         df[df.memory == 1 * G], title="Plot Comparison: 8G Mem/8 CPU/16 Data Partition"
#     )
#     .mark_bar()
#     .encode(
#         y="name:N",
#         x=alt.X("elapsed", title="Elapsed (s)"),
#         color="name",
#         tooltip="elapsed",
#         row="nrow:Q",
#         column=alt.Column("format:O", title="Data Format"),
#     )
#     .resolve_scale(x="independent")
# )
# chartBar
##
# pdf = df.pivot_table(
#     index=["Mem", "CPU", "Dataset", "Partition", "Row", "Col", "Mode"],
#     columns="Func",
#     values="Elapsed",
# ).reset_index()


##
